<a href="https://colab.research.google.com/github/kuoootina/aop113/blob/main/EX05_04_Reply_Template_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pyngrok flask_ngrok # for colab
!pip install -q flask line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 7.2 MB/s eta 0:00:00


In [ ]:
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
    ngrok.connect(5000)
    run_with_ngrok(app)

from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    TemplateMessage, ConfirmTemplate, MessageAction,
    CarouselTemplate,
    CarouselColumn,
    URIAction,
    PostbackAction
)

app = Flask(__name__)

@app.route("/", methods=['GET'])
def index():
    return "hello"

configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text
        if action == 'confirm':
            template=ConfirmTemplate(
                  text="你確定要進行這個動作嗎？",
                  actions=[
                      MessageAction(label="是", text="是"),
                      MessageAction(label="否", text="否")
                  ]
                )
            reply = TemplateMessage(
                alt_text="這是確認視窗",
                template=template
        )
        elif action == 'carousel':
          carousel_template = CarouselTemplate(
              columns=[
                  CarouselColumn(
                      thumbnail_image_url='https://example.com/image1.jpg',
                      title='商品 A',
                      text='這是商品 A 的描述，價格 $100。',
                      actions=[
                          URIAction(label='查看詳情', uri='https://example.com/productA'),
                          MessageAction(label="投票", text="我投商品A一票")
                      ]
                ),
                  CarouselColumn(
                      thumbnail_image_url='https://example.com/image2.jpg',
                      title='商品 B',
                      text='這是商品 B 的描述，價格 $100。',
                      actions=[
                          URIAction(label='查看詳情', uri='https://example.com/productB'),
                          MessageAction(label="投票", text="我投商品B一票")
                      ]
                )
            ]
          )
          reply = TemplateMessage(
              alt_text="這是確認視窗",
              template=CarouselTemplate()
          )
        else:
          reply = TextMessage(text='Please type "confirm"!')

        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                   reply
                ]
            )
        )



ngrok_start() # for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://606f-35-221-164-151.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
